In [2]:
#!/usr/bin/python 
# -*- coding: utf-8 -*-

import torch
import torchvision
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
import os
from PIL import Image
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from PIL import Image
import torchxrayvision as xrv
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from sklearn.metrics import roc_auc_score
import re

torch.cuda.empty_cache()

In [48]:
device = 'cuda'
transform = transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                        xrv.datasets.XRayResizer(224)])
bs = 4

train_set = xrv.datasets.COVID19_Dataset(
            imgpath='covid_data/images',
            csvpath='covid_data/train_n.csv',
            transform=transform)

print(f'\nCovid Chest x-ray stats train set stats:\n{train_set}',flush=True)

{'ARDS': {0.0: 205, 1.0: 4},
 'Bacterial Pneumonia': {0.0: 205, 1.0: 4},
 'COVID-19': {0.0: 152, 1.0: 57},
 'MERS': {0.0: 209},
 'No Finding': {0.0: 208, 1.0: 1},
 'Pneumonia': {0.0: 141, 1.0: 68},
 'SARS': {0.0: 206, 1.0: 3},
 'Streptococcus': {0.0: 205, 1.0: 4},
 'Viral Pneumonia': {0.0: 149, 1.0: 60}}

Covid Chest x-ray stats train set stats:
COVID19_Dataset num_samples=209


In [49]:
val_set = xrv.datasets.COVID19_Dataset(
            imgpath='covid_data/images',
            csvpath='covid_data/validation_n.csv',
            transform=transform)

print(f'Covid Chest x-ray stats validation set stats:\n\n{val_set}',flush=True)

{'ARDS': {0.0: 48},
 'Bacterial Pneumonia': {0.0: 48},
 'COVID-19': {0.0: 38, 1.0: 10},
 'MERS': {0.0: 48},
 'No Finding': {0.0: 48},
 'Pneumonia': {0.0: 30, 1.0: 18},
 'SARS': {0.0: 40, 1.0: 8},
 'Streptococcus': {0.0: 48},
 'Viral Pneumonia': {0.0: 30, 1.0: 18}}
Covid Chest x-ray stats validation set stats:

COVID19_Dataset num_samples=48


In [50]:
test_set = xrv.datasets.COVID19_Dataset(
            imgpath='covid_data/images',
            csvpath='covid_data/test_n.csv',
            transform=transform)
print(f'Covid Chest x-ray stats test set stats:\n\n{test_set}',flush=True)\

{'ARDS': {0.0: 42},
 'Bacterial Pneumonia': {0.0: 40, 1.0: 2},
 'COVID-19': {0.0: 33, 1.0: 9},
 'MERS': {0.0: 42},
 'No Finding': {0.0: 42},
 'Pneumonia': {0.0: 31, 1.0: 11},
 'SARS': {0.0: 42},
 'Streptococcus': {0.0: 40, 1.0: 2},
 'Viral Pneumonia': {0.0: 33, 1.0: 9}}
Covid Chest x-ray stats test set stats:

COVID19_Dataset num_samples=42


In [68]:
def load_train_data():
    train_loader = torch.utils.data.DataLoader(train_set, batch_size = bs, num_workers = 0, shuffle=True)
    return train_loader

def load_val_data():
    val_loader = torch.utils.data.DataLoader(val_set, batch_size = bs, num_workers = 0, shuffle=False)
    return val_loader

def load_test_data():
    test_loader = torch.utils.data.DataLoader(test_set, batch_size = bs, num_workers = 0, shuffle=False)
    return test_loader

In [26]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
#         print('lam',lam)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
#     print(pred)
#     print(y_a)
#     print('criterion',criterion(pred, y_a))
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [61]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

alpha = 0.6

def train(optimizer, epoch):
    
    model.train()
    train_loader = load_train_data()
    
    train_loss = 0
    train_correct = 0
    
    for batch_index, batch_samples in enumerate(train_loader):
        
        # move data to device
        data, target = batch_samples["PA"].to(device), batch_samples["lab"].to(device)
        data, targets_a, targets_b, lam = mixup_data(data, target.long()[:, 2],
                                                       alpha, use_cuda=True)
        
        
        optimizer.zero_grad()
        output = model(data)
        
        criteria = nn.CrossEntropyLoss()
        loss = criteria(output, target.long()[:, 2])
#         loss = mixup_criterion(criteria, output, targets_a, targets_b, lam)
        train_loss += criteria(output, target.long()[:, 2])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.long()[:, 2].view_as(pred)).sum().item()
    
        # Display progress and write to tensorboard
        if batch_index % bs == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
            f = open('model_result/DenseNet_alpha_{}.txt'.format(alpha), 'a+')
            f.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
            f.write('\n')
            f.close()
#             niter = epoch*len(train_loader)+batch_index
#             writer.add_scalar('Train/Loss', loss.data, niter)
    
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))
    f = open('model_result/DenseNet_alpha_{}.txt'.format(alpha), 'a+')
    f.write('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))
    f.write('\n')
    f.close()

In [57]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

alpha = 0.6

def train(optimizer, epoch):
    
    model.train()
    train_loader = load_train_data()
    
    train_loss = 0
    train_correct = 0
    
    for batch_index, batch_samples in enumerate(train_loader):
        
        # move data to device
        data, target = batch_samples["PA"].to(device), batch_samples["lab"].to(device)
        data, targets_a, targets_b, lam = mixup_data(data, target.long()[:, 2],
                                                       alpha, use_cuda=True)
        
        
        optimizer.zero_grad()
        output = model(data)
        
#         loss = nn.functional.nll_loss(output, target.long()[:, 2])
        criteria = nn.CrossEntropyLoss()
        loss = mixup_criterion(criteria, output, targets_a, targets_b, lam)
        train_loss += criteria(output, target.long()[:, 2])
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.long()[:, 2].view_as(pred)).sum().item()
    
        # Display progress and write to tensorboard
        if batch_index % bs == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
            f = open('model_result/DenseNet_alpha_{}.txt'.format(alpha), 'a+')
            f.write('Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}'.format(
                epoch, batch_index, len(train_loader),
                100.0 * batch_index / len(train_loader), loss.item()/ bs))
            f.write('\n')
            f.close()
#             niter = epoch*len(train_loader)+batch_index
#             writer.add_scalar('Train/Loss', loss.data, niter)
    
    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))
    f = open('model_result/DenseNet_alpha_{}.txt'.format(alpha), 'a+')
    f.write('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        train_loss/len(train_loader.dataset), train_correct, len(train_loader.dataset),
        100.0 * train_correct / len(train_loader.dataset)))
    f.write('\n')
    f.close()

In [85]:
def test(epoch):
    
    model.eval()
    test_loss = 0
    correct = 0
    test_loader = load_test_data()
    results = []
    
    TP = 0
    TN = 0
    FN = 0
    FP = 0
    
    
    criteria = nn.CrossEntropyLoss()
    # Don't update model
    with torch.no_grad():
        tpr_list = []
        fpr_list = []
        
        predlist=[]
        targetlist=[]
        # Predict
        for batch_index, batch_samples in enumerate(test_loader):
            
            data, target = batch_samples["PA"].to(device), batch_samples["lab"].to(device)
            output = model(data)
            
            
            test_loss += criteria(output, target.long()[:, 2])
            pred = output.argmax(dim=1, keepdim=True)
#             print(pred)
#             print(target.long()[:, 2].view_as(pred))
            correct += pred.eq(target.long()[:, 2].view_as(pred)).sum().item()
            TP += ((pred == 1) & (target.long()[:, 2].view_as(pred).data == 1)).cpu().sum()
            TN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
#             # FN    predict 0 label 1
            FN += ((pred == 0) & (target.long()[:, 2].view_as(pred) == 1)).cpu().sum()
#             # FP    predict 1 label 0
            FP += ((pred == 1) & (target.long()[:, 2].view_as(pred) == 0)).cpu().sum()
            print(TP,TN,FN,FP)
            
            
#             print(output[:,1].cpu().numpy())
#             print((output[:,1]+output[:,0]).cpu().numpy())
#             predcpu=(output[:,1].cpu().numpy())/((output[:,1]+output[:,0]).cpu().numpy())
            targetcpu=target.long()[:, 2].cpu().numpy()
            predlist=np.append(predlist, pred.cpu().numpy())
            targetlist=np.append(targetlist,targetcpu)
           
        print('epoch',epoch)
        print('pred',predlist)
        print('target',targetlist)
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP.item() / (TP + FP).item()
        print('precision',p)
        p = TP.item() / (TP + FP).item()
        r = TP.item() / (TP + FN).item()
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN).item() / (TP + TN + FP + FN).item()
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, predlist)
        print('AUC', AUC)
       
    test_loss /= len(test_loader.dataset)
    

    # Display results
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100.0 * correct / len(test_loader.dataset)))
    
    
    return r, p, acc, AUC, targetlist, predlist
    
    # Write to tensorboard
#     writer.add_scalar('Test Accuracy', 100.0 * correct / len(test_loader.dataset), epoch)

In [42]:
class DenseNet121(nn.Module):
    """Model modified.

    The architecture of our model is the same as standard DenseNet121
    except the classifier layer which has an additional sigmoid function.

    """
    def __init__(self, out_size):
        super(DenseNet121, self).__init__()
        self.densenet121 = torchvision.models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, out_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.densenet121(x)
        return x

In [43]:
#model define

device = 'cuda'

CKPT_PATH = 'model.pth.tar'
N_CLASSES = 14

DenseNet121 = DenseNet121(N_CLASSES).cuda()

In [44]:
CKPT_PATH = './CheXNet/model.pth.tar'

if os.path.isfile(CKPT_PATH):
    print("=> loading checkpoint")
    checkpoint = torch.load(CKPT_PATH)        
    state_dict = checkpoint['state_dict']
    remove_data_parallel = False


    pattern = re.compile(
        r'^(.*denselayer\d+\.(?:norm|relu|conv))\.((?:[12])\.(?:weight|bias|running_mean|running_var))$')
    for key in list(state_dict.keys()):
        match = pattern.match(key)
        new_key = match.group(1) + match.group(2) if match else key
        new_key = new_key[7:] if remove_data_parallel else new_key
        new_key = new_key[7:]
        state_dict[new_key] = state_dict[key]
        del state_dict[key]


    DenseNet121.load_state_dict(checkpoint['state_dict'])
    print("=> loaded checkpoint")
#     print(densenet121)
else:
    print("=> no checkpoint found")
    
DenseNet121.densenet121.classifier._modules['0'] = nn.Linear(in_features=1024, out_features=2, bias=True)
DenseNet121.densenet121.features.conv0 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
print(DenseNet121)
model = DenseNet121.to(device)

=> loading checkpoint
=> loaded checkpoint
DenseNet121(
  (densenet121): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
       

In [86]:
# 1dense

r_list = []
p_list = []
acc_list = []
AUC_list = []
# TP = 0
# TN = 0
# FN = 0
# FP = 0
test_loader = load_test_data()
vote_pred = np.zeros((1,len(test_loader.dataset)))

#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum = 0.9)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
scheduler = StepLR(optimizer, step_size=1)

total_epoch = 1000
for epoch in range(1, total_epoch+1):
    train(optimizer, epoch)
    
    r, p, acc, AUC, targetlist, predlist = test(epoch)
    vote_pred = vote_pred + predlist 
    
    p_list.append(p)
    r_list.append(r)
    acc_list.append(acc)
    AUC_list.append(AUC)

    # Save model
    if epoch == total_epoch:
       torch.save(model.state_dict(), "model_backup/new_split_DenseNet_{}.pt".format(epoch))  

    if epoch % 10 == 0:
        r_ave = sum(r_list)/len(r_list)
        p_ave = sum(p_list)/len(p_list)
        acc_ave = sum(acc_list)/len(acc_list)
        AUC_ave = sum(AUC_list)/len(AUC_list)
        F1_ave = 2 * r_ave * p_ave / (r_ave + p_ave)
        r_list = []
        p_list = []
        acc_list = []
        AUC_list = []
        vote_pred[vote_pred <= 5] = 0
        vote_pred[vote_pred > 5] = 1
        
        TP = ((vote_pred == 1) & (targetlist == 1)).sum()
        TN = ((vote_pred == 0) & (targetlist == 0)).sum()
        FN = ((vote_pred == 0) & (targetlist == 1)).sum()
        FP = ((vote_pred == 1) & (targetlist == 0)).sum()
        
        
        print('TP=',TP,'TN=',TN,'FN=',FN,'FP=',FP)
        print('TP+FP',TP+FP)
        p = TP / (TP + FP)
        print('precision',p)
        p = TP / (TP + FP)
        r = TP / (TP + FN)
        print('recall',r)
        F1 = 2 * r * p / (r + p)
        acc = (TP + TN) / (TP + TN + FP + FN)
        print('F1',F1)
        print('acc',acc)
        AUC = roc_auc_score(targetlist, predlist)
        print('AUC', AUC)
        
        
        f = open('model_result/DenseNet_alpha_{}.txt'.format(alpha), 'a+')
        f.write('precision, recall, F1, acc= \n')
        f.writelines(str(p))
        f.writelines('\n')
        f.writelines(str(r))
        f.writelines('\n')
        f.writelines(str(F1))
        f.writelines('\n')
        f.writelines(str(acc))
        f.writelines('\n')
        f.close()
        
        
        vote_pred = np.zeros((1,len(test_loader.dataset)))
        print('vote_pred',vote_pred)
        print('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},\
average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r_ave, p_ave, F1_ave, acc_ave, AUC_ave))

        f = open('model_result/new_split_DenseNet_alpha_{}.txt'.format(alpha), 'a+')
        f.write('\n The epoch is {}, average recall: {:.4f}, average precision: {:.4f},\
average F1: {:.4f}, average accuracy: {:.4f}, average AUC: {:.4f}'.format(
        epoch, r_ave, p_ave, F1_ave, acc_ave, AUC_ave))
        f.close()
# state = {'epoch': epoch + 1, 'state_dict': model.state_dict(),
#                  'optimizer': optimizer.state_dict(), 'scheduler' : scheduler}
# torch.save(state, "model_backup/AlexNetAdamState")



Train Epoch: 1 [0/53 (0%)]	Train Loss: 0.078340
Train Epoch: 1 [4/53 (8%)]	Train Loss: 0.078841
Train Epoch: 1 [8/53 (15%)]	Train Loss: 0.199139
Train Epoch: 1 [12/53 (23%)]	Train Loss: 0.202816
Train Epoch: 1 [16/53 (30%)]	Train Loss: 0.080314
Train Epoch: 1 [20/53 (38%)]	Train Loss: 0.078423
Train Epoch: 1 [24/53 (45%)]	Train Loss: 0.079666
Train Epoch: 1 [28/53 (53%)]	Train Loss: 0.081593
Train Epoch: 1 [32/53 (60%)]	Train Loss: 0.078447
Train Epoch: 1 [36/53 (68%)]	Train Loss: 0.078323
Train Epoch: 1 [40/53 (75%)]	Train Loss: 0.126214
Train Epoch: 1 [44/53 (83%)]	Train Loss: 0.171130
Train Epoch: 1 [48/53 (91%)]	Train Loss: 0.195550
Train Epoch: 1 [52/53 (98%)]	Train Loss: 0.078318

Train set: Average loss: 0.1208, Accuracy: 174/209 (83%)

tensor(1) tensor(2) tensor(0) tensor(1)
tensor(2) tensor(2) tensor(3) tensor(1)
tensor(5) tensor(2) tensor(4) tensor(1)
tensor(5) tensor(6) tensor(4) tensor(1)
tensor(5) tensor(10) tensor(4) tensor(1)
tensor(5) tensor(13) tensor(4) tensor(2)
tens

Train Epoch: 6 [0/53 (0%)]	Train Loss: 0.082283
Train Epoch: 6 [4/53 (8%)]	Train Loss: 0.083012
Train Epoch: 6 [8/53 (15%)]	Train Loss: 0.196030
Train Epoch: 6 [12/53 (23%)]	Train Loss: 0.078495
Train Epoch: 6 [16/53 (30%)]	Train Loss: 0.078337
Train Epoch: 6 [20/53 (38%)]	Train Loss: 0.078367
Train Epoch: 6 [24/53 (45%)]	Train Loss: 0.078726
Train Epoch: 6 [28/53 (53%)]	Train Loss: 0.203220
Train Epoch: 6 [32/53 (60%)]	Train Loss: 0.079573
Train Epoch: 6 [36/53 (68%)]	Train Loss: 0.079151
Train Epoch: 6 [40/53 (75%)]	Train Loss: 0.202352
Train Epoch: 6 [44/53 (83%)]	Train Loss: 0.150168
Train Epoch: 6 [48/53 (91%)]	Train Loss: 0.081886
Train Epoch: 6 [52/53 (98%)]	Train Loss: 0.078321

Train set: Average loss: 0.1197, Accuracy: 175/209 (84%)

tensor(1) tensor(1) tensor(0) tensor(2)
tensor(3) tensor(1) tensor(2) tensor(2)
tensor(7) tensor(1) tensor(2) tensor(2)
tensor(7) tensor(5) tensor(2) tensor(2)
tensor(7) tensor(9) tensor(2) tensor(2)
tensor(7) tensor(12) tensor(2) tensor(3)
tenso

Train Epoch: 11 [0/53 (0%)]	Train Loss: 0.079851
Train Epoch: 11 [4/53 (8%)]	Train Loss: 0.078707
Train Epoch: 11 [8/53 (15%)]	Train Loss: 0.160578
Train Epoch: 11 [12/53 (23%)]	Train Loss: 0.199839
Train Epoch: 11 [16/53 (30%)]	Train Loss: 0.192118
Train Epoch: 11 [20/53 (38%)]	Train Loss: 0.193835
Train Epoch: 11 [24/53 (45%)]	Train Loss: 0.188789
Train Epoch: 11 [28/53 (53%)]	Train Loss: 0.078335
Train Epoch: 11 [32/53 (60%)]	Train Loss: 0.078326
Train Epoch: 11 [36/53 (68%)]	Train Loss: 0.079629
Train Epoch: 11 [40/53 (75%)]	Train Loss: 0.197621
Train Epoch: 11 [44/53 (83%)]	Train Loss: 0.080807
Train Epoch: 11 [48/53 (91%)]	Train Loss: 0.078938
Train Epoch: 11 [52/53 (98%)]	Train Loss: 0.078320

Train set: Average loss: 0.1229, Accuracy: 173/209 (83%)

tensor(1) tensor(2) tensor(0) tensor(1)
tensor(4) tensor(2) tensor(1) tensor(1)
tensor(8) tensor(2) tensor(1) tensor(1)
tensor(8) tensor(6) tensor(1) tensor(1)
tensor(8) tensor(10) tensor(1) tensor(1)
tensor(8) tensor(13) tensor(1) 

Train Epoch: 16 [0/53 (0%)]	Train Loss: 0.078325
Train Epoch: 16 [4/53 (8%)]	Train Loss: 0.148399
Train Epoch: 16 [8/53 (15%)]	Train Loss: 0.078926
Train Epoch: 16 [12/53 (23%)]	Train Loss: 0.082186
Train Epoch: 16 [16/53 (30%)]	Train Loss: 0.078332
Train Epoch: 16 [20/53 (38%)]	Train Loss: 0.078571
Train Epoch: 16 [24/53 (45%)]	Train Loss: 0.080196
Train Epoch: 16 [28/53 (53%)]	Train Loss: 0.078944
Train Epoch: 16 [32/53 (60%)]	Train Loss: 0.079233
Train Epoch: 16 [36/53 (68%)]	Train Loss: 0.159159
Train Epoch: 16 [40/53 (75%)]	Train Loss: 0.079246
Train Epoch: 16 [44/53 (83%)]	Train Loss: 0.104749
Train Epoch: 16 [48/53 (91%)]	Train Loss: 0.193539
Train Epoch: 16 [52/53 (98%)]	Train Loss: 0.078324

Train set: Average loss: 0.1138, Accuracy: 178/209 (85%)

tensor(1) tensor(3) tensor(0) tensor(0)
tensor(4) tensor(3) tensor(1) tensor(0)
tensor(6) tensor(3) tensor(3) tensor(0)
tensor(6) tensor(7) tensor(3) tensor(0)
tensor(6) tensor(11) tensor(3) tensor(0)
tensor(6) tensor(14) tensor(3) 

Train Epoch: 21 [0/53 (0%)]	Train Loss: 0.078350
Train Epoch: 21 [4/53 (8%)]	Train Loss: 0.100258
Train Epoch: 21 [8/53 (15%)]	Train Loss: 0.078771
Train Epoch: 21 [12/53 (23%)]	Train Loss: 0.152063
Train Epoch: 21 [16/53 (30%)]	Train Loss: 0.078347
Train Epoch: 21 [20/53 (38%)]	Train Loss: 0.078462
Train Epoch: 21 [24/53 (45%)]	Train Loss: 0.201235
Train Epoch: 21 [28/53 (53%)]	Train Loss: 0.078339
Train Epoch: 21 [32/53 (60%)]	Train Loss: 0.078401
Train Epoch: 21 [36/53 (68%)]	Train Loss: 0.089352
Train Epoch: 21 [40/53 (75%)]	Train Loss: 0.200338
Train Epoch: 21 [44/53 (83%)]	Train Loss: 0.202151
Train Epoch: 21 [48/53 (91%)]	Train Loss: 0.078339
Train Epoch: 21 [52/53 (98%)]	Train Loss: 0.078320

Train set: Average loss: 0.1174, Accuracy: 177/209 (85%)

tensor(1) tensor(2) tensor(0) tensor(1)
tensor(4) tensor(2) tensor(1) tensor(1)
tensor(4) tensor(2) tensor(5) tensor(1)
tensor(4) tensor(6) tensor(5) tensor(1)
tensor(4) tensor(10) tensor(5) tensor(1)
tensor(4) tensor(13) tensor(5) 

Train Epoch: 26 [0/53 (0%)]	Train Loss: 0.078399
Train Epoch: 26 [4/53 (8%)]	Train Loss: 0.079197
Train Epoch: 26 [8/53 (15%)]	Train Loss: 0.086662
Train Epoch: 26 [12/53 (23%)]	Train Loss: 0.078489
Train Epoch: 26 [16/53 (30%)]	Train Loss: 0.078565
Train Epoch: 26 [20/53 (38%)]	Train Loss: 0.078338
Train Epoch: 26 [24/53 (45%)]	Train Loss: 0.078364
Train Epoch: 26 [28/53 (53%)]	Train Loss: 0.202174
Train Epoch: 26 [32/53 (60%)]	Train Loss: 0.181517
Train Epoch: 26 [36/53 (68%)]	Train Loss: 0.078329
Train Epoch: 26 [40/53 (75%)]	Train Loss: 0.078534
Train Epoch: 26 [44/53 (83%)]	Train Loss: 0.141062
Train Epoch: 26 [48/53 (91%)]	Train Loss: 0.203764
Train Epoch: 26 [52/53 (98%)]	Train Loss: 0.078319

Train set: Average loss: 0.1082, Accuracy: 185/209 (89%)

tensor(1) tensor(3) tensor(0) tensor(0)
tensor(4) tensor(3) tensor(1) tensor(0)
tensor(6) tensor(3) tensor(3) tensor(0)
tensor(6) tensor(7) tensor(3) tensor(0)
tensor(6) tensor(11) tensor(3) tensor(0)
tensor(6) tensor(14) tensor(3) 

Train Epoch: 31 [0/53 (0%)]	Train Loss: 0.150392
Train Epoch: 31 [4/53 (8%)]	Train Loss: 0.082831
Train Epoch: 31 [8/53 (15%)]	Train Loss: 0.203154
Train Epoch: 31 [12/53 (23%)]	Train Loss: 0.195663
Train Epoch: 31 [16/53 (30%)]	Train Loss: 0.078484
Train Epoch: 31 [20/53 (38%)]	Train Loss: 0.082827
Train Epoch: 31 [24/53 (45%)]	Train Loss: 0.079043
Train Epoch: 31 [28/53 (53%)]	Train Loss: 0.193488
Train Epoch: 31 [32/53 (60%)]	Train Loss: 0.078403
Train Epoch: 31 [36/53 (68%)]	Train Loss: 0.127720
Train Epoch: 31 [40/53 (75%)]	Train Loss: 0.079767
Train Epoch: 31 [44/53 (83%)]	Train Loss: 0.078331
Train Epoch: 31 [48/53 (91%)]	Train Loss: 0.197941
Train Epoch: 31 [52/53 (98%)]	Train Loss: 0.078322

Train set: Average loss: 0.1207, Accuracy: 174/209 (83%)

tensor(1) tensor(2) tensor(0) tensor(1)
tensor(4) tensor(2) tensor(1) tensor(1)
tensor(5) tensor(2) tensor(4) tensor(1)
tensor(5) tensor(6) tensor(4) tensor(1)
tensor(5) tensor(10) tensor(4) tensor(1)
tensor(5) tensor(13) tensor(4) 

Train Epoch: 36 [0/53 (0%)]	Train Loss: 0.078326
Train Epoch: 36 [4/53 (8%)]	Train Loss: 0.078385
Train Epoch: 36 [8/53 (15%)]	Train Loss: 0.140880
Train Epoch: 36 [12/53 (23%)]	Train Loss: 0.078517
Train Epoch: 36 [16/53 (30%)]	Train Loss: 0.078433
Train Epoch: 36 [20/53 (38%)]	Train Loss: 0.078437
Train Epoch: 36 [24/53 (45%)]	Train Loss: 0.080106
Train Epoch: 36 [28/53 (53%)]	Train Loss: 0.201026
Train Epoch: 36 [32/53 (60%)]	Train Loss: 0.203187
Train Epoch: 36 [36/53 (68%)]	Train Loss: 0.078553
Train Epoch: 36 [40/53 (75%)]	Train Loss: 0.079481
Train Epoch: 36 [44/53 (83%)]	Train Loss: 0.202760
Train Epoch: 36 [48/53 (91%)]	Train Loss: 0.140854
Train Epoch: 36 [52/53 (98%)]	Train Loss: 0.078322

Train set: Average loss: 0.1210, Accuracy: 175/209 (84%)

tensor(0) tensor(2) tensor(1) tensor(1)
tensor(3) tensor(2) tensor(2) tensor(1)
tensor(6) tensor(2) tensor(3) tensor(1)
tensor(6) tensor(6) tensor(3) tensor(1)
tensor(6) tensor(10) tensor(3) tensor(1)
tensor(6) tensor(13) tensor(3) 

ZeroDivisionError: float division by zero